In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, ConvE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'convE_transductive_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
batch_size = 512

In [8]:
model = ConvE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 5383030
Space occupied: 21532120 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory convE_transductive_io created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper,
        batch_size = batch_size
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/162 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.970497605241375
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00024276310016962025
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 9.317583668932503e-05
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00016796946842947266
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00024276311160065236
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 9.317582589574157e-05
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.00016796946874819696
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0002427630988923802
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 9.317581880077345e-05
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00016796945884657685
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.

Training batches on cuda:0:   0%|          | 0/162 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 2.818886269757777
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00016676878756056383
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 7.145951366946724e-05
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00011911415061501555
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0001667687902227044
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 7.145951531128958e-05
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0001191141491290182
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0001667687857129113
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 7.1459511977205e-05
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00011911414884505814
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.head

...

Step: 50
Metric: loss = 0.0009240898816418018
Step: 50
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04194876887667705
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.06818748414841852
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.055068126512547784
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.04194876551628113
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.06818748265504837
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.05506812781095504
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.041948768265988416
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.06818748383561016
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.05506812605079929
Metric: validation.head.optimistic.hits_at_1 = 0.007979758660957572
Metric: validation.tail.optimistic.hits_at_1 = 0.01605683145192682
Metric: validation.both.optimistic.hits_at_1 = 0.012

Training batches on cuda:0:   0%|          | 0/162 [00:00<?, ?batch/s]

Step: 51
Metric: loss = 0.0008327617697320005
Step: 51
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04743036447991066
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.07560875204273898
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.06151955826132482
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0474303662776947
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0756087526679039
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.06151955947279931
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.04743036431882536
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.07560875184170907
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.061519558080267205
Metric: validation.head.optimistic.hits_at_1 = 0.012553522771506422
Metric: validation.tail.optimistic.hits_at_1 = 0.023939275982872713
Metric: validation.both.optimistic.hits_at_1 = 0.01824

Training batches on cuda:0:   0%|          | 0/162 [00:00<?, ?batch/s]

Step: 52
Metric: loss = 0.0006664647105621335
Step: 52
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.05127770371835733
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.07094258574617922
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.06111014473226828
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.051277704536914825
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.07094258069992065
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.06111014261841774
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.05127770258558653
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0709425850045351
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.061110143795060815
Metric: validation.head.optimistic.hits_at_1 = 0.011385753211366291
Metric: validation.tail.optimistic.hits_at_1 = 0.01605683145192682
Metric: validation.both.optimistic.hits_at_1 = 0.0137

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   3.970498
1   2.818886
2   1.846845
3   1.208633
4   0.869401
5   0.724829
6   0.674330
7   0.623165
8   0.552430
9   0.473028
10  0.394813
11  0.322430
12  0.252910
13  0.180177
14  0.118392
15  0.078813
16  0.055822
17  0.041249
18  0.031653
19  0.026458
20  0.021217
21  0.017368
22  0.014878
23  0.012307
24  0.010698
25  0.009129
26  0.008124
27  0.007167
28  0.006482
29  0.005816
30  0.004935
31  0.004284
32  0.003739
33  0.003736
34  0.003166
35  0.002764
36  0.002474
37  0.002494
38  0.002129
39  0.001678
40  0.001973
41  0.001659
42  0.001552
43  0.001394
44  0.001108
45  0.001204
46  0.001238
47  0.001108
48  0.001006
49  0.000924
50  0.000833
51  0.000666


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.074512   0.074512     0.074512
hits_at_1                     0.034156   0.034156     0.034156
hits_at_3                     0.072382   0.072382     0.072382
hits_at_5                     0.102996   0.102996     0.102996
hits_at_10                    0.155217   0.155217     0.155217
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.205176   0.205176     0.205176
hits_at_1                     0.100034   0.100034     0.100034
hits_at_3                     0.220036   0.220036     0.220036
hits_at_5                     0.304100   0.304100     0.304100
hits_at_10                    0.440505   0.440505     0.440505
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.139844   0.139844     0.139844
hits_at_1                     0.067095   0.067095     0.067095
hits_at_3                     0.146209  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.055225   0.055225     0.055225
hits_at_1                     0.019755   0.019755     0.019755
hits_at_3                     0.049630   0.049630     0.049630
hits_at_5                     0.074835   0.074835     0.074835
hits_at_10                    0.122908   0.122908     0.122908
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.093175   0.093175     0.093175
hits_at_1                     0.034255   0.034255     0.034255
hits_at_3                     0.092254   0.092254     0.092254
hits_at_5                     0.137408   0.137408     0.137408
hits_at_10                    0.212729   0.212729     0.212729
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.074200   0.074200     0.074200
hits_at_1                     0.027005   0.027005     0.027005
hits_at_3                     0.070942  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.060614   0.060614     0.060614
hits_at_1                     0.024316   0.024316     0.024316
hits_at_3                     0.055590   0.055590     0.055590
hits_at_5                     0.080440   0.080440     0.080440
hits_at_10                    0.128689   0.128689     0.128689
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.113596   0.113596     0.113596
hits_at_1                     0.054596   0.054596     0.054596
hits_at_3                     0.117449   0.117449     0.117449
hits_at_5                     0.153617   0.153617     0.153617
hits_at_10                    0.221823   0.221823     0.221823
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.087105   0.087105     0.087105
hits_at_1                     0.039456   0.039456     0.039456
hits_at_3                     0.086519  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['batch size'] = batch_size

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   
21                      batch size   

                                                value  
0                                              cuda:0  
1            

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)